In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import pandas_profiling
%matplotlib inline

In [2]:
train_ = pd.read_csv('./data/train.csv',index_col=0)
test_ = pd. read_csv('./data/test.csv',index_col=0)
sample_submission = pd.read_csv('./data/sample_submission.csv', index_col=0)

데이터 설명

-     대전지역에서 측정한 실내외 19곳의 센서데이터와, 주변 지역의 기상청 공공데이터를 semi-비식별화하여 제공합니다. 
-     센서는 온도를 측정하였습니다. 
-     모든 데이터는 시간 순으로 정렬 되어 있으며 10분 단위 데이터 입니다. 
-     예측 대상(target variable)은 Y18입니다. 

train.csv 

-     30일 간의 기상청 데이터 (X00 ~ X39) 및 센서데이터 (Y00~Y17)
-     이후 3일 간의 기상청 데이터 (X00~X39) 및 센서데이터 (Y18)

test.csv 
-     train.csv 기간 이후 80일 간의 기상청 데이터 (X00~X39)

sample_submission.csv
-     제출 양식 예시

X00 X07 X28 X31 기온  
X01 X06 X22 X27 기압  
X02 X03 X18 X24 풍속  
X04 X10 X21 X36 누적강수량  
X05 X08 X09 X23 해면기압  
X11 X14 X16 X19 누적일사량  
X12, X20, X30, X37 습도
X13 X15 X17 X25 풍향  

Y00~17 센서측정온도  

In [3]:
train=train_.copy()
test=test_.copy()

### Y00 ~ Y17 >>> [0:4320]: Value, [4321:]: NaN, Y_index: [41:58]
### Y18 >>> [0:4320]: NaN, [4321:]: Value, Y_index: [59]

In [4]:
tp=0
k=0

In [5]:
for i in range(40,58):
    if i != 46 | i != 47 | i != 52:
        tp += train.iloc[0:4320,i]
        k+=1

In [6]:
train.shape

(4752, 59)

In [7]:
train.iloc[0:4320,58]=tp/k

In [8]:
train.head()

,X00,X01,X02,X03,X04,X05,X06,X07,X08,X09,...,Y09,Y10,Y11,Y12,Y13,Y14,Y15,Y16,Y17,Y18
id,,,,,,,,,,,,,,,,,,,,,
0,9.7,988.8,1.2,0.6,0.0,1009.3,989.6,12.2,1009.9,1009.8,...,7.0,7.5,7.0,9.0,10.0,9.5,9.0,8.0,9.0,10.9375
1,9.3,988.9,1.7,1.9,0.0,1009.3,989.6,12.1,1010.0,1009.9,...,6.5,7.5,7.0,8.5,10.0,9.5,9.0,7.5,9.0,10.7500
2,9.4,989.0,1.1,2.3,0.0,1009.2,989.7,12.1,1010.1,1010.1,...,6.5,7.5,6.5,8.0,9.5,9.5,8.5,7.5,8.5,10.5000
3,9.4,988.9,1.5,0.7,0.0,1009.2,989.6,12.0,1010.0,1010.0,...,6.0,7.0,6.0,8.0,9.5,9.0,8.5,7.5,8.5,10.3750
4,9.2,988.9,0.8,1.7,0.0,1009.2,989.7,12.0,1010.1,1010.0,...,6.0,7.0,6.0,7.5,9.5,9.0,8.5,7.5,8.5,10.1875


In [46]:
x=train.iloc[:,1:40]
y=train.iloc[:,-1]

In [47]:
from sklearn.model_selection import train_test_split, KFold, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit

x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, shuffle=False) #random_state=0)


In [56]:
test_x=test.iloc[:,1:40]

In [88]:
from sklearn.preprocessing import StandardScaler, RobustScaler

scaler2=StandardScaler()

scaler2.fit(x_train)
x_train=scaler2.transform(x_train)
x_test=scaler2.transform(x_test)

test_x=scaler2.transform(test_x)


In [95]:
x1_train=np.array(x_train)
y1_train=np.array(y_train)

In [96]:
from keras.layers import Dense, Input, Dropout, Activation, BatchNormalization, Concatenate, Conv1D, Flatten, MaxPooling1D
from keras.models import Model, load_model
from keras import optimizers, callbacks
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K
from swa.keras import SWA
from keras import initializers
from keras_radam import RAdam


In [45]:

inps = Input(shape = (39,))

x= Dense(250)(inps)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(250)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(250)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(350)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)
x= Dropout(0.25)(x)

x= Dense(250)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)

x= Dense(250)(x)
x= BatchNormalization()(x)
x= Activation('elu')(x)
x= Dropout(0.25)(x)

outs=Dense(1,name='output')(x)

models = Model(inputs=inps, outputs=outs)
#models.compile(RAdam(),loss='mse')
models.compile(loss='mse',optimizer=Adam(lr=8.e-6))
models.summary()


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 39)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 250)               10000     
_________________________________________________________________
batch_normalization_22 (Batc (None, 250)               1000      
_________________________________________________________________
activation_22 (Activation)   (None, 250)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 250)               62750     
_________________________________________________________________
batch_normalization_23 (Batc (None, 250)               1000      
_________________________________________________________________
activation_23 (Activation)   (None, 250)               0   

In [46]:
x1_train.shape,y1_train.shape

((3801, 39), (3801,))

In [ ]:
hist = models.fit(x1_train, y1_train, batch_size= 8, epochs=2500, verbose=2, validation_split=0.1)

Train on 3420 samples, validate on 381 samples
Epoch 1/2500
 - 9s - loss: 560.1214 - val_loss: 537.5782
Epoch 2/2500
 - 5s - loss: 536.0247 - val_loss: 518.6982
Epoch 3/2500
 - 6s - loss: 524.4653 - val_loss: 520.9567
Epoch 4/2500
 - 6s - loss: 514.4760 - val_loss: 514.5514
Epoch 5/2500
 - 6s - loss: 505.5729 - val_loss: 492.0170
Epoch 6/2500
 - 6s - loss: 496.7790 - val_loss: 487.3487
Epoch 7/2500
 - 5s - loss: 490.0698 - val_loss: 486.1502
Epoch 8/2500
 - 6s - loss: 482.6536 - val_loss: 465.9215
Epoch 9/2500
 - 6s - loss: 472.7655 - val_loss: 464.3294
Epoch 10/2500
 - 5s - loss: 466.3514 - val_loss: 462.4528
Epoch 11/2500
 - 6s - loss: 456.7756 - val_loss: 445.4202
Epoch 12/2500
 - 6s - loss: 448.6570 - val_loss: 440.3721
Epoch 13/2500
 - 5s - loss: 442.3964 - val_loss: 438.4216
Epoch 14/2500
 - 6s - loss: 431.8392 - val_loss: 436.1587
Epoch 15/2500
 - 5s - loss: 425.5663 - val_loss: 424.0065
Epoch 16/2500
 - 5s - loss: 417.0042 - val_loss: 411.9859
Epoch 17/2500
 - 5s - loss: 407.97